# Interpreting line by line Michael Nelson code on his tutorial and adding explainatory comments. Some lines of code needed to be modified to incorporate changes in Theano modules.

In [2]:
# ALL BOLD COMMENTS ARE 'TO-DOs' ; COME BACK TO COMPLETE THEM
# Note: This code is an updated version of what is provided on Michael Nelson's website,
# as some of the modules have been moved.

# Standard library
import cPickle
import gzip

# Third-party libraries
import numpy as np
import theano
import theano.tensor as T
from theano.tensor.nnet import conv
from theano.tensor.nnet import softmax
from theano.tensor import shared_randomstreams
from theano.tensor.signal import pool

# Activation functions for neurons
def linear(z): return z
def ReLU(z): return T.maximum(0.0, z)
from theano.tensor.nnet import sigmoid
from theano.tensor import tanh

#### Constants
GPU = True
if GPU:
    print "Trying to run under a GPU.  If this is not desired, then modify "+\
        "network3.py\nto set the GPU flag to False."
    try: theano.config.device = 'gpu'
    except: pass # it's already set
    theano.config.floatX = 'float32'
else:
    print "Running with a CPU.  If this is not desired, then the modify "+\
        "network3.py to set\nthe GPU flag to True."

#### Load the MNIST data
def load_data_shared(filename="../data/mnist.pkl.gz"):
    f = gzip.open(filename, 'rb')
    training_data, validation_data, test_data = cPickle.load(f)
    f.close()
    def shared(data):

        shared_x = theano.shared(
            np.asarray(data[0], dtype=theano.config.floatX), borrow=True)
        shared_y = theano.shared(
            np.asarray(data[1], dtype=theano.config.floatX), borrow=True)
        return shared_x, T.cast(shared_y, "int32")
    return [shared(training_data), shared(validation_data), shared(test_data)]

Using gpu device 0: GeForce GT 645M (CNMeM is disabled, cuDNN not available)


Trying to run under a GPU.  If this is not desired, then modify network3.py
to set the GPU flag to False.


In [2]:
class fullyConnectedLayer(object):

    def __init__(self, nIn, nOut, activationFn, pDropout=0.0):
        self.nIn = nIn
        self.nOut = nOut
        self.activationFn = activationFn
        self.pDropout = pDropout
        # Initialize weights and biases
        # this weight and bias initialization are designed for the sigmoid activation function
        # (as discussed earlier). Ideally, we'd initialize the weights and biases somewhat differently
        # for activation functions such as the tanh and rectified linear function
        self.w = theano.shared(
            np.asarray(
                np.random.normal(
                    loc=0.0, scale=np.sqrt(1.0/nOut), size=(nIn, nOut)),
                dtype=theano.config.floatX),
            name='w', borrow=True)
        self.b = theano.shared(
            np.asarray(np.random.normal(loc=0.0, scale=1.0, size=(nOut,)),
                       dtype=theano.config.floatX),
            name='b', borrow=True)
        self.params = [self.w, self.b]

    # this function will be used to compute the input of the FullyConnectedLayer
    # and to compute the corresponding output
    
    def setInput(self, inpt, inptDropout, miniBatchSize):
        # COME BACK TO COMPLETE IT LATER
        self.inpt = inpt.reshape((miniBatchSize, self.nIn))
        
        self.yOut = T.argax( self.output ,axis = 1)

# this is not needed except in the last layer
#     def accuracy(self, y):
#         "Return the accuracy for the mini-batch."
#         return T.mean(T.eq(y, self.yOut))

In [1]:
class netowrk(object):
    
    def __init__(self, layers, miniBatchSize):
        """
        It takes a list of layers which describe the network architecture
        as the 'layers' argument and a value for 'miniBatchSize' to be used while 
        training using stochastic gradient descent """
        self.layers = layers
        self.miniBatchSize = miniBatchSize
        
        # a nested loop for bundling up all the parameters from different layers
        self.params = [params for j in self.layers for params in j.params]
        
        # symbolic matrix variable 'X' will contain the training examples in a 
        # mini batch, where the columns are the features and rows are training examples
        # symbolic vector variable 'y' will contain the corresponding target values
        self.X = T.matrix('X')
        self.y = T.ivector('y')
        inputLayer = self.layers[0]
        inputLayer.setInput(self.X, self.X, self.miniBatchSize)
        
        # propagate through the network architecture
        for j in range(1, len(layers)):
            previousLayer, currentLayer = layers[j-1], layers[j]
            currentLayer.setInput(previousLayer.output, previousLayer.outputDropout, self.miniBatchSize)
        
        # there will be an output variable which will be the final output of our CNN
        self.output = self.layers[-1].output
        self.outputDropout = self.layers[-1].outputDropout
        
    def sgd(self, epochs, trainingData, validationData, testData, eta, regularizationParameter = 0.0):
        """Train the data using mini-batch stochastic gradient descent"""
        
        # get features and target values separately
        trainX, trainY = trainingData
        testX, testY = testData
        validationX, validationY = validationData
        
        # compute the number of miniBatches for training, validation and testing
        
        # CURRENTLY I DON'T KNOW THE 'type' OF 'traininData', 'testData', AND 'validationData'
        # SO I'M USING 'len' TO GET ITS SIZE
        numTrainingBatches = len(trainingData)/miniBatchSize
        numValidationBatches = len(validationData)/miniBatchSize
        numTestBatches = len(testData)/miniBatchSize
        
        # symbolic computations
        # 1. regularization term in cost ; NOTE: .sum() is for numpy arrays, sum(___) is for python iterables
        l2NormalizationSquared = sum([(layer.w**2).sum() for layer in self.layers])
        cost = self.layers[-1].cost(self) + 0.5*lmbda*l2NormalizationSquared/numTrainingBatches
        grads = T.grad(cost, self.params)
        updates = [(param, param-eta*grad) for param in zip(self.params, grads)]
        
        # functions to train mini-batches and to calculate accuracy in validation
        # and test mini-batches
        i = T.lscalar() # mini batch number
        trainMb = theano.function([i], cost, updates = updates, givens = {
                self.X : trainX[i*miniBatchSize : (i+1)*miniBatchSize],
                self.y : testY[i*miniBatchSize : (i+1)*miniBatchSize]
            }
        )
        
        validateMb = theano.function([i], self.layers[-1].accuracy(self.y), givens = {
                self.X : validationX[i*miniBatchSize : (i+1)*miniBatchSize],
                self.y : validationY[i*miniBatchSize : (i+1)*miniBatchSize]
            }
        )
        
        testMb = theano.function([i], self.layers[-1].accuracy(self.y), givens = {
                self.X : trainX[i*miniBatchSize : (i+1)*miniBatchSize],
                self.y : testY[i*miniBatchSize : (i+1)*miniBatchSize]
            }
        )
        
        bestValidationAccuracy = 0.0
        for epoch in xrange(epochs):
            for miniBatch in xrange(numTrainingBatches):
                iteration = numTrainingBatches*epoch + miniBatch
                if iteration%100 == 0:
                    print "This is iteration number {0}".format(iteration)
                cost = trainMb([miniBatch])
                if (iteration+1)%numTrainingBatches == 0:
                    validationAccuracy = np.mean([validateMb([j]) for j in numValidationBatches])
                    if validationAccuracy > bestValidationAccuracy:
                        bestIteration = iteration
                        print "This is the best validation accuracy so far : {0}, obtained in iteration number = {1}".format(validationAccuracy, iteration)
                        testAccuracy = np.mean([testMb(j) for j in numTestBatches])
                        print "The corresponding test accuracy is {0}".format(testAccuracy)
                        
        print "Finished training network"
        print "The best validation accuracy was obtained {0}, was obtained in iteration number {1}".format(validationAccuracy, iteration)
        print "The corresponding test accuracy was {0}".format(testAccuracy)

In [3]:
class convPoolLayer(object):
    """
    will implement a convolutional + pooling layer.
    """
    def __init__(self, filterShape, imageShape, poolSize = (2,2), activationFn = sigmoid):
        
        # DON'T WORRY ABOUT WHY THESE TWO PARAMETERS ARE THE WAY THEY ARE. 
        # YOU DID NOT HAVE TO COME UP WITH THIS. IT'S OKAY. IT'S BEING GOVERNED BY
        # THEANO IMPLEMENTATION DETAILS
        
        # `filterShape` is a tuple of length 4, whose entries are the number
        # of filters, the number of input feature maps, the filter height, and the
        # filter width.
        self.filterShape = filterShape
        
        # 'image_shape` is a tuple of length 4, whose entries are the
        # mini-batch size, the number of input feature maps, the image
        # height, and the image width.
        self.imageShape = imageShape
        
        self.poolSize = poolSize
        self.activationFn = activationFn
        
        # initialize weights and biases
        self.w = theano.shared(np.asarray(np.random.normal(loc = 0, scale = np.sqrt(1.0/nOut), 
                                                size = filterShape), dtype = theano.config.floatX), borrow = True)
        self.b = theano.shared(np.asarray(np.random.normal(loc = 0, scale = 1.0, 
                                                size = (filterShape[0],)), dtype = theano.config.floatX), borrow = True)
        self.params = [self.w, self.b]
        
    def setInput(self, inpt, inptDropout, miniBatchSize):
        self.inpt = inpt.reshape(imageShape)
        
        # this output will be a 4-tuple ( batch size, # output feature maps, width , height )
        convOut = conv.conv2d(input = self. inpt, filters = self.w, filter_shape = self.filterShape, 
                              input_shape = self.imageShape)
        
        # 'pool.pool_2d' : Takes as input a N-D tensor, where N >= 2. It downscales the input image by the 
        # specified factor, by keeping only the maximum value of non-overlapping patches of size (ds[0], ds[1]).
        # Max pooling will be done over the 2 last dimensions.
        
        # I AM NOT VERY CLEAR ABOUT HOW PADDING WORKS AND HOW THIS 'ignore_border' parameter is to 
        # be used
        pooledOut = pool.pool_2d(input = convOut, ds = self.poolSize, ignore_border=True, mode = 'max')
        self.output = self.activationFn(pooledOut + self.b.dimshuffle('x', 0, 'x', 'x'))
        self.outputDropout = self.output # no dropout in the convolutional layers

In [ ]:
class softMaxLayer(object):

    def __init__(self, nIn, nOut, pDropout=0.0):
        self.nIn = nIn
        self.nOut = nOut
        self.pDropout = pDropout
        # Initialize weights and biases
        self.w = theano.shared(
            np.zeros((nIn, nOut), dtype=theano.config.floatX),
            name='w', borrow=True)
        self.b = theano.shared(
            np.zeros((nOut,), dtype=theano.config.floatX),
            name='b', borrow=True)
        self.params = [self.w, self.b]

    def setInpt(self, inpt, inptDropout, miniBatchSize):
        self.inpt = inpt.reshape((mini_batch_size, self.nIn))
        self.output = softmax((1-self.pDropout)*T.dot(self.inpt, self.w) + self.b)
        self.yOut = T.argmax(self.output, axis=1)
        self.inptDropout = dropoutLayer(
            inptDropout.reshape((miniBatchSize, self.nIn)), self.pDropout)
        self.outputDropout = softmax(T.dot(self.inptDropout, self.w) + self.b)

    def cost(self, net):
        "Return the log-likelihood cost."
        return -T.mean(T.log(self.outputDropout)[T.arange(net.y.shape[0]), net.y])

    def accuracy(self, y):
        "Return the accuracy for the mini-batch."
        return T.mean(T.eq(y, self.yOut))

In [ ]:
def dropout_layer(layer, pDropout):
    srng = shared_randomstreams.RandomStreams(
        np.random.RandomState(0).randint(999999))
    mask = srng.binomial(n=1, p=1-pDropout, size=layer.shape)
    return layer*T.cast(mask, theano.config.floatX)